In [1]:
from utils.ds_clean import clean_ds
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split


2024-04-04 19:06:47.622221: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-04 19:06:47.622574: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 19:06:47.625975: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 19:06:47.666091: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 19:06:48.485171: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
df1 = pd.read_csv("data/aapl_5m_train.csv")
df_5min = clean_ds(df1)


In [ ]:
df_5min

In [3]:
close_data_buy = df_5min[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'X_t-4' ,'X_t-5','RSI', 'Y_BUY']]
close_data_buy = close_data_buy.dropna()

In [ ]:
close_data_buy.to_csv('./data/close_data_buy_5.csv', index=False)

In [4]:
close_data_sell = df_5min[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'X_t-4' ,'X_t-5','RSI', 'Y_SELL']]
close_data_sell = close_data_sell.dropna()

In [5]:
close_data_sell

,Timestamp,Close,X_t-1,X_t-2,X_t-3,X_t-4,X_t-5,RSI,Y_SELL
27,1609778700,129.413299,129.390106,129.690002,129.910003,129.895004,129.996398,28.060866,True
28,1609779000,129.148498,129.413299,129.390106,129.690002,129.910003,129.895004,26.485360,True
29,1609779300,128.820007,129.148498,129.413299,129.390106,129.690002,129.910003,24.701195,True
30,1609779600,128.580001,128.820007,129.148498,129.413299,129.390106,129.690002,23.501640,True
31,1609779900,128.351699,128.580001,128.820007,129.148498,129.413299,129.390106,22.427254,True
...,...,...,...,...,...,...,...,...,...
39574,1672432800,129.054992,129.039993,129.020004,128.910003,128.940002,128.890106,56.509324,False
39575,1672433100,129.380004,129.054992,129.039993,129.020004,128.910003,128.940002,60.700323,False
39576,1672433400,129.764404,129.380004,129.054992,129.039993,129.020004,128.910003,64.854367,False
39577,1672433700,129.949996,129.764404,129.380004,129.054992,129.039993,129.020004,66.620919,False


In [ ]:
close_data_sell.to_csv('./data/close_data_sell_5.csv', index=False)

In [ ]:
x = close_data_buy.drop(columns=['Y_BUY']).values
y = close_data_buy[['Y_BUY']].values

In [ ]:
features = x.shape[1]
features

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

In [ ]:
x_train = x_train.reshape(-1, features, 1)
x_test = x_test.reshape(-1, features, 1)

In [ ]:
dnn = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Dense(units=256, activation='relu', input_shape=(features,)),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')

])    

In [ ]:
dnn.compile(optimizer='adam', 
            loss='binary_crossentropy', 
            metrics=['accuracy'])

In [ ]:
dnn.summary()

In [ ]:
dnn.fit(x_train, y_train, epochs=50)